## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
# from config import g_key
import sys, os
sys.path.append('../')

from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,0,Krasnotorka,48.6820,37.5327,58.53,87,29,7.29,UA,scattered clouds,2021-06-07 04:20:37
1,1,Sterling,39.0062,-77.4286,80.60,79,90,4.61,US,overcast clouds,2021-06-07 04:18:00
2,2,Hilo,19.7297,-155.0900,77.29,78,90,4.61,US,heavy intensity rain,2021-06-07 04:20:37
3,3,Butaritari,3.0707,172.7902,82.94,72,23,7.87,KI,light rain,2021-06-07 04:20:37
4,4,Wewak,-3.5534,143.6268,81.66,78,100,4.50,PG,overcast clouds,2021-06-07 04:20:38


In [ ]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

In [ ]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

In [ ]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count() == preferred_cities_df.notnull().sum()

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Not applicable, nothing to drop since my df doesn't have any empty rows.

In [ ]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country","Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": "39.0062, -77.4286",
    "language": "en"
}

# 6b. Iterate through the hotel DataFrame.
for idx, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    params["location"] = f"{lat}, {lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    try: 
        hotels = requests.get(base_url, params=params).json()
        # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.at[idx,"Hotel Name"] = hotels["results"][0]["name"]
    except:
        print(f"Hotel not found in City: {city}")
        pass



In [ ]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df
clean_hotel_df = clean_hotel_df.set_index("Hotel Name")
clean_hotel_df = clean_hotel_df.drop("", axis=0)
clean_hotel_df = clean_hotel_df.reset_index()
clean_hotel_df.head()

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig